In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
# borehole_df = pd.read_csv('../../Documents/borehole_data/subsurface_layer_data_joined.csv')

/var/folders/ct/x2tsq5qj4v512jqb_k8l1jgw0000gp/T/ipykernel_64517/1530089489.py:1: DtypeWarning: Columns (9,10,11,12,13,30,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  borehole_df = pd.read_csv('../../Documents/borehole_data/subsurface_layer_data_joined.csv')


In [ ]:
# output_csv_path = '../data/full_lithology_training_data.csv'

In [12]:
min_samples_for_stratify = 2
test_size_proportion = 0.25
random_seed = 42

In [5]:
lat_col = 'LATITUDE'
lon_col = 'LONGITUDE'
collar_elev_col = 'ELEVATION_FT'
top_depth_col = 'TOP_DEPTH_FT'
bottom_depth_col = 'BOTTOM_DEPTH_FT'
lithology_col = 'SYMBOL_LITHOLOGY'

midpoint_depth_col = 'Midpoint_Depth_ft'
midpoint_elev_col = 'Midpoint_Elevation_ft'

feature_cols = [lat_col, lon_col, midpoint_depth_col, midpoint_elev_col]

In [3]:
borehole_df.shape

(410677, 39)

In [6]:
initial_rows = borehole_df.shape[0]
borehole_df.dropna(subset=[collar_elev_col, top_depth_col, bottom_depth_col, lithology_col, lat_col, lon_col], inplace=True)
rows_after_nan_drop = borehole_df.shape[0]
if initial_rows > rows_after_nan_drop:
        print(f"Dropped {initial_rows - rows_after_nan_drop} rows with missing essential data.")

Dropped 70801 rows with missing essential data.


In [7]:
print("Calculating midpoint depth and elevation...")
borehole_df[midpoint_depth_col] = (borehole_df[top_depth_col] + borehole_df[bottom_depth_col]) / 2
borehole_df[midpoint_elev_col] = borehole_df[collar_elev_col] - borehole_df[midpoint_depth_col]

Calculating midpoint depth and elevation...


In [9]:
print("\nChecking class counts before filtering...")
class_counts = borehole_df[lithology_col].value_counts()
print("Original Class Distribution (Top 20):")
print(class_counts.head(20)) # Print top common classes

classes_to_remove = class_counts[class_counts < min_samples_for_stratify].index.tolist()


Checking class counts before filtering...
Original Class Distribution (Top 20):
SYMBOL_LITHOLOGY
Silty sand              123744
Sand                     43099
Sandy silt               29899
Silt                     22086
Topsoil / vegetation     22029
Gravelly sand            17388
Clayey silt              11088
Gravel                    9785
Asphalt / concrete        9301
Silty clay                8808
Clay                      7233
Silty gravel              6433
Fill                      5589
Peat                      5280
Clayey sand               3891
Undefined                 3354
Sedimentary bedrock       2606
Gravelly silt             2299
Sandy clay                2098
Sandy gravel              1613
Name: count, dtype: int64


In [11]:
if classes_to_remove:
    print(f"\nFound classes with less than {min_samples_for_stratify} members: {classes_to_remove}. These will be removed for stratification.")
    df_filtered = borehole_df[~borehole_df[lithology_col].isin(classes_to_remove)].copy()
    print(f"Data shape after removing rare classes: {df_filtered.shape}")
    print("Updated Class Distribution (Top 20):")
    print(df_filtered[lithology_col].value_counts().head(20))
else:
    print("\nNo classes with less than 2 members found. Proceeding with split.")
    df_filtered = borehole_df.copy()


No classes with less than 2 members found. Proceeding with split.


In [16]:
df_filtered.to_csv(output_csv_path, index=False)

In [14]:
if df_filtered.shape[0] == 0:
        raise ValueError("No data remaining after cleaning and removing rare classes. Cannot perform split.")

# --- Separate features (X) and target (y) on the filtered data ---
X = df_filtered[feature_cols]
y = df_filtered[lithology_col]

print(f"\nFeatures shape (X): {X.shape}")
print(f"Target shape (y): {y.shape}")
print(f"Target variable unique values after cleaning: {y.unique().shape[0]} unique lithology types.")


# --- Perform the train-test split ---
print(f"\nSplitting data into training ({1 - test_size_proportion:.0%}) and testing ({test_size_proportion:.0%})...")

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=test_size_proportion,
    random_state=random_seed,
    stratify=y # Stratify should now work
)

print("Data split complete.")
print(f"Training features shape (X_train): {X_train.shape}")
print(f"Testing features shape (X_test): {X_test.shape}")
print(f"Training target shape (y_train): {y_train.shape}")
print(f"Testing target shape (y_test): {y_test.shape}")

# Check class distribution in train/test sets
print("\nClass distribution comparison (Training vs. Testing):")
train_dist = y_train.value_counts(normalize=True)
test_dist = y_test.value_counts(normalize=True)
dist_comparison = pd.DataFrame({'Train': train_dist, 'Test': test_dist})
print(dist_comparison.head()) # Print comparison for top classes


Features shape (X): (339876, 4)
Target shape (y): (339876,)
Target variable unique values after cleaning: 29 unique lithology types.

Splitting data into training (75%) and testing (25%)...
Data split complete.
Training features shape (X_train): (254907, 4)
Testing features shape (X_test): (84969, 4)
Training target shape (y_train): (254907,)
Testing target shape (y_test): (84969,)

Class distribution comparison (Training vs. Testing):
                         Train      Test
SYMBOL_LITHOLOGY                        
Silty sand            0.364086  0.364086
Sand                  0.126807  0.126811
Sandy silt            0.087969  0.087973
Silt                  0.064981  0.064988
Topsoil / vegetation  0.064816  0.064812


In [30]:
lithology_group_map = {
    '(Metamorphic bedrock)': 'Bedrock',
    'Plutonic bedrock': 'Bedrock',
    'Sedimentary bedrock': 'Bedrock',
    'Volcanic bedrock': 'Bedrock',
    'Undifferentiated rock': 'Bedrock',

    'Debris': 'Anthropogenic Fill/Debris',
    'Fill': 'Anthropogenic Fill/Debris',
    'Asphalt / concrete': 'Asphalt / concrete',
    'Topsoil / vegetation': 'Topsoil / vegetation',

    'Peat': 'Peat',

    'Clay': 'Clayey Soils',
    'Silty clay': 'Clayey Soils',
    'Sandy clay': 'Clayey Soils',
    'Gravelly clay': 'Clayey Soils',

    'Silt': 'Silty Soils',
    'Clayey silt': 'Silty Soils',
    'Sandy silt': 'Silty Soils',
    'Gravelly silt': 'Silty Soils',

    'Sand': 'Sand (with Fines)', # Naming implies potential fines, but sand is dominant
    'Silty sand': 'Sand (with Fines)',
    'Clayey sand': 'Sand (with Fines)',

    'Gravel': 'Gravel (with Fines)', # Naming implies potential fines, but gravel is dominant
    'Sandy gravel': 'Gravel (with Fines)',
    'Silty gravel': 'Gravel (with Fines)',
    'Clayey gravel': 'Gravel (with Fines)',
    'Cobbles / boulders': 'Gravel (with Fines)', # Or potentially a separate 'Coarse Aggregate' group

    # 'Gravelly sand': 'Mixed Sand & Gravel', # Keeping this potentially separate as a common mix

    'Undefined': 'Undefined/Remove', # Mark for removal
    'Volcanic ash': 'Undefined/Remove' # Mark for removal due to rarity
}

In [31]:
grouped_lithology_col = 'Grouped_Lithology'

In [32]:
df_filtered[grouped_lithology_col] = df_filtered[lithology_col].map(lithology_group_map)

In [20]:
df_filtered = df_filtered[df_filtered[grouped_lithology_col] != 'Undefined/Remove'].copy()

In [34]:
initial_rows = df_filtered.shape[0]
df_filtered.dropna(subset=[collar_elev_col, top_depth_col, bottom_depth_col, grouped_lithology_col, lat_col, lon_col], inplace=True)
rows_after_nan_drop = df_filtered.shape[0]
if initial_rows > rows_after_nan_drop:
        print(f"Dropped {initial_rows - rows_after_nan_drop} rows with missing essential data.")

In [33]:
print("Checking grouped class counts before filtering...")
grouped_class_counts = df_filtered[grouped_lithology_col].value_counts()
print("Grouped Class Distribution:")
print(grouped_class_counts)

Checking grouped class counts before filtering...
Grouped Class Distribution:
Grouped_Lithology
Sand (with Fines)            170734
Silty Soils                   65372
Topsoil / vegetation          22029
Clayey Soils                  18585
Gravel (with Fines)           18356
Asphalt / concrete             9301
Anthropogenic Fill/Debris      6235
Peat                           5280
Bedrock                        3222
Name: count, dtype: int64


In [35]:
df_filtered.to_csv('../data/full_grouped_lithology_training_data.csv', index=False)

In [ ]:
surface_slope_col = 'Surface_Slope' # Name of the new column from DEM features file
surface_aspect_col = 'Surface_Aspect' # Name of the new column from DEM features file

# Define features for the model (updated to include DEM features)
# feature_cols = [lat_col, lon_col, midpoint_depth_col, midpoint_elev_col, surface_slope_col, surface_aspect_col]

# New target variable name (grouped lithology)
grouped_lithology_col = 'Grouped_Lithology'

# Define the proportion of data to use for testing
test_size_proportion = 0.25

# Define a random state for reproducibility
random_seed = 42

# Minimum samples required for a grouped class to be included in training/testing with stratification
min_samples_for_stratify = 10 # Increased minimum threshold as we have more data

# --- Lithology Grouping Dictionary ---
# Define how original lithologies map to new grouped categories
# **Adjust this dictionary based on your refined grouping strategy**
lithology_group_map = {
    '(Metamorphic bedrock)': 'Bedrock',
    'Plutonic bedrock': 'Bedrock',
    'Sedimentary bedrock': 'Bedrock',
    'Volcanic bedrock': 'Bedrock',
    'Undifferentiated rock': 'Bedrock',

    'Asphalt / concrete': 'Anthropogenic',
    'Debris': 'Anthropogenic',
    'Fill': 'Anthropogenic',
    'Topsoil / vegetation': 'Anthropogenic', # Or maybe 'Near-Surface' depending on context

    'Peat': 'Peat', # Often kept separate due to unique properties

    'Clay': 'Clayey Soils',
    'Silty clay': 'Clayey Soils',
    'Sandy clay': 'Clayey Soils',
    'Gravelly clay': 'Clayey Soils',

    'Silt': 'Silty Soils',
    'Clayey silt': 'Silty Soils',
    'Sandy silt': 'Silty Soils',
    'Gravelly silt': 'Silty Soils',

    'Sand': 'Sand (with Fines)', # Naming implies potential fines, but sand is dominant
    'Silty sand': 'Sand (with Fines)',
    'Clayey sand': 'Sand (with Fines)',

    'Gravel': 'Gravel (with Fines)', # Naming implies potential fines, but gravel is dominant
    'Sandy gravel': 'Gravel (with Fines)',
    'Silty gravel': 'Gravel (with Fines)',
    'Clayey gravel': 'Gravel (with Fines)',
    'Cobbles / boulders': 'Gravel (with Fines)', # Or potentially a separate 'Coarse Aggregate' group

    # 'Gravelly sand': 'Mixed Sand & Gravel', # Keeping this potentially separate as a common mix

    'Undefined': 'Undefined/Remove', # Mark for removal
    'Volcanic ash': 'Undefined/Remove' # Mark for removal due to rarity
}


    # --- Data Cleaning and Feature Engineering ---
    # Merge main data with DEM features
    print(f"Merging main data with DEM features on '{borehole_id_col}'...")
    # Ensure boreholes can be matched - assumes borehole_id_col is consistent
    # If matching on Lat/Lon, might need rounding or different merge strategy
    df_merged = pd.merge(df_large, df_dem[[borehole_id_col, surface_slope_col, surface_aspect_col]], on=borehole_id_col, how='left')
    print(f"Shape after merge: {df_merged.shape}")
    # Handle cases where no DEM data was found for a borehole (will result in NaNs in new columns)
    if df_merged[[surface_slope_col, surface_aspect_col]].isnull().sum().sum() > 0:
         print("\nWarning: Missing DEM feature values found after merge. Consider imputing or dropping rows.")
         print(df_merged[[surface_slope_col, surface_aspect_col]].isnull().sum())
         # For now, let's drop rows missing DEM features to keep it simple
         merge_initial_rows = df_merged.shape[0]
         df_merged.dropna(subset=[surface_slope_col, surface_aspect_col], inplace=True)
         merge_rows_after_drop = df_merged.shape[0]
         if merge_initial_rows > merge_rows_after_drop:
              print(f"Dropped {merge_initial_rows - merge_rows_after_drop} rows missing DEM features.")


    # Drop rows where essential data for layer definition or original lithology is missing
    initial_rows = df_merged.shape[0]
    df_merged.dropna(subset=[collar_elev_col, top_depth_col, bottom_depth_col, original_lithology_col, lat_col, lon_col], inplace=True)
    rows_after_nan_drop = df_merged.shape[0]
    if initial_rows > rows_after_nan_drop:
         print(f"Dropped {initial_rows - rows_after_nan_drop} rows with missing essential data.")


    # Calculate Midpoint Depth and Midpoint Elevation
    print("Calculating midpoint depth and elevation...")
    df_merged[midpoint_depth_col] = (df_merged[top_depth_col] + df_merged[bottom_depth_col]) / 2
    df_merged[midpoint_elev_col] = df_merged[collar_elev_col] - df_merged[midpoint_depth_col]

    # --- Apply Lithology Grouping ---
    print("\nApplying lithology grouping...")
    df_merged[grouped_lithology_col] = df_merged[original_lithology_col].map(lithology_group_map).fillna('Other/Unknown') # Map & handle potential unmapped values

    # --- Handle Rare/Undefined Grouped Classes for Stratification ---
    print("Checking grouped class counts before filtering...")
    grouped_class_counts = df_merged[grouped_lithology_col].value_counts()
    print("Grouped Class Distribution:")
    print(grouped_class_counts)

    # Remove the 'Undefined/Remove' group explicitly
    df_filtered = df_merged[df_merged[grouped_lithology_col] != 'Undefined/Remove'].copy()

    # Also remove any remaining grouped classes with less than min_samples_for_stratify members
    grouped_class_counts_after_removal = df_filtered[grouped_lithology_col].value_counts()
    rare_grouped_classes = grouped_class_counts_after_removal[grouped_class_counts_after_removal < min_samples_for_stratify].index.tolist()

    if rare_grouped_classes:
        print(f"\nFound grouped classes with less than {min_samples_for_stratify} members after initial removal: {rare_grouped_classes}. These will be removed for stratification.")
        df_filtered = df_filtered[~df_filtered[grouped_lithology_col].isin(rare_grouped_classes)].copy()
        print(f"Data shape after removing rare grouped classes: {df_filtered.shape}")
        print("Final Grouped Class Distribution:")
        print(df_filtered[grouped_lithology_col].value_counts())
    else:
        print("\nNo grouped classes with less than minimum samples found after initial removal. Proceeding with split.")


    # Ensure there's still data left to split
    if df_filtered.shape[0] == 0:
         raise ValueError("No data remaining after cleaning, merging, and removing rare grouped classes. Cannot perform split.")

    # --- Separate features (X) and target (y) on the filtered data ---
    X = df_filtered[feature_cols]
    y = df_filtered[grouped_lithology_col]

    print(f"\nFeatures shape (X): {X.shape}")
    print(f"Target shape (y): {y.shape}")
    print(f"Target variable unique values after cleaning: {y.unique().shape[0]} unique grouped lithology types.")


    # --- Perform the train-test split ---
    print(f"\nSplitting data into training ({1 - test_size_proportion:.0%}) and testing ({test_size_proportion:.0%})...")

    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=test_size_proportion,
        random_state=random_seed,
        stratify=y # Stratify should now work on grouped lithologies
    )

    print("Data split complete.")
    print(f"Training features shape (X_train): {X_train.shape}")
    print(f"Testing features shape (X_test): {X_test.shape}")
    print(f"Training target shape (y_train): {y_train.shape}")
    print(f"Testing target shape (y_test): {y_test.shape}")

    # Check class distribution in train/test sets
    print("\nGrouped Class distribution comparison (Training vs. Testing):")
    train_dist = y_train.value_counts(normalize=True)
    test_dist = y_test.value_counts(normalize=True)
    dist_comparison = pd.DataFrame({'Train': train_dist, 'Test': test_dist})
    print(dist_comparison)


    # --- Proceed to Model Training with X_train, X_test, y_train, y_test ---
    # The next code block will use these variables.

except FileNotFoundError as e:
    print(f"Error: Input file not found - {e}. Please check file paths.")
except KeyError as e:
    print(f"Error: Column '{e}' not found in one of the dataframes. Please check column names in configuration and input files.")
except ValueError as e:
    print(f"Data or Configuration Error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")